## Installing Libraries & Importing Packages

In [ ]:
!pip install --upgrade --force-reinstall thefuzz[speeedup]==0.19.0
!pip install python-Levenshtein

     |████████████████████████████████| 50 kB 4.3 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149844 sha256=b1b8005ced64a2ebcb2e7b2300f9b5770a66da6eed6f9da4ba10458d604a3c92
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
import math

from thefuzz import fuzz
from thefuzz import process
from IPython.display import display

##Loading & Displaying the Dataframes

In [ ]:
# Loading candidate datframe
candidate_df = pd.read_csv("/content/sample_data/User_Dataset_for_building_recommendation_system.csv")
# Removing column name "Unnamed"
candidate_df = candidate_df.loc[:, ~candidate_df.columns.str.contains('^Unnamed: 0')]
# Convertting strings to list 
candidate_df["AllSkills"] = candidate_df["AllSkills"].apply(eval)

# Loading skill simmilarity datframe
skill_sim_df = pd.read_csv("/content/sample_data/All_skills_with_similarity_dataset.csv")
# Removing column name "Unnamed"
skill_sim_df = skill_sim_df.loc[:, ~skill_sim_df.columns.str.contains('^Unnamed: 0')]

skill_sim_df = skill_sim_df.replace('_',' ', regex=True)
skill_sim_df = skill_sim_df.replace('-',' ', regex=True)
skill_sim_df = skill_sim_df.replace('-',' ', regex=True)
# Feature Scaling
skill_sim_df[['Simmilarity/Correlation']] = StandardScaler().fit_transform(skill_sim_df[['Simmilarity/Correlation']])

# Loading candidate datframe
jobs_desc_df = pd.read_csv("/content/sample_data/SDE_MLE_DS_skillsExtracted_Sen_RolesOffered_companyname.csv")
# Removing column name "Unnamed"
jobs_desc_df = jobs_desc_df.loc[:, ~jobs_desc_df.columns.str.contains('^Unnamed: 0')]
jobs_desc_df["skills_required"] = jobs_desc_df["skills_required"].apply(eval)

In [ ]:
# Loading full profilee of the candidate
# This data would only be used at the end to display full profile of the recommended candidates

candidate_full_profile = pd.read_csv("/content/sample_data/All_candidate_full_profiles.csv")
# Removing column name "Unnamed"
candidate_full_profile = candidate_full_profile.loc[:, ~candidate_full_profile.columns.str.contains('^Unnamed: 0')]
candidate_full_profile.head()

,userId,Name,Current_Company,Job_Title,Job_Date_Range,Company_2,Job_Title2,Job_Date_Range2,Job_Description_2,University,Degree,Degree_Name,AllSkills
0,425562397,Yumiao Zhang,Instacart,Senior Software Engineer,Oct 2021 – Present,Amazon,Software Engineer,Feb 2016 – Oct 2021,Build reporting experience and analytical syst...,Washington University in St. Louis,Master's,Electrical and Electronics Engineering,"HTML, CSS, JavaScript, Software Engineering, B..."
1,215180440,Arjun Gopalan,Google,Senior Software Engineer,May 2018 – Present,Tintri,Senior Member of Technical Staff,Sep 2014 – May 2018,- Key member of the engineering team involved ...,Stanford University,Master's,Computer Science,"Distributed Systems, Storage Systems, Algorith..."
2,111186163,Haoming Chen,Google,Senior Software Engineer,Jul 2016 – Present,Apple,SDE Intern,Jun 2015 – Sep 2015,Deep-learning based image quality assessment,University of Washington,PhD,"Video/Image Compression, Computer Vision","Computer Vision, Machine Learning, Computer Gr..."
3,207600550,Fan Di,Google,Senior Software Engineer,Jun 2018 – Present,MathWorks,Software Engineer Intern,May 2017 – May 2018,Jan 2018 - May 2018 (Part-time 20 hrs/week in ...,Harvard University,Master's,Landscape Architecture,"Java, Algorithms, Python, Data Structures, Art..."
4,44528135,Neha Gupta,Google,Senior Software Engineer,Nov 2019 – Present,Google,Software Engineer,Feb 2015 – Nov 2019,Adwords Frontend Software Engineer,Georgia Institute of Technology,Bachelor’s,Computer Science,"Machine Learning, C, Linux, Artificial Intelli..."


In [ ]:
# disaply dataframe
candidate_df.head()

,userId,Years_of_Exper,Degree,AllSkills
0,425562397,Senior,Master's,"[amazon ec2, software engineering, xcode, aws ..."
1,215180440,Senior,Master's,"[python, c, c++, matlab, algorithm, html, java]"
2,111186163,Senior,PhD,"[machine learning, python, c, c/c++, neural ne..."
3,207600550,Mid,Master's,"[python, c, javascript, matlab, mapreduce, com..."
4,44528135,Senior,Bachelor’s,"[data mining, natural language processing, mac..."


In [ ]:
# disaply dataframe
skill_sim_df.head()

,Skill_1,Skill_2,Simmilarity/Correlation
0,jquery,bootstrap,-0.396930
1,jquery,restful,-0.397198
2,jquery,salesforce,-0.397749
3,jquery,github,-0.398066
4,jquery,cryptography,-0.398077


In [ ]:
# disaply dataframe
jobs_desc_df.head()

,companyName,company_offeredRole,Seniority Level,skills_required
0,Amazon Data Services UK Ltd,Software Engineer,Entry,"[agile, algorithms, master, java, computer sci..."
1,Applied Nanodetectors Ltd,Software Engineer,Mid,"[bachelor, deep learning, algorithm, python, o..."
2,Jane Street,Software Engineer,Entry,[]
3,CITADEL ENTERPRISE EUROPE LIMITED,Software Engineer,Entry,"[probability, master, research, phd, computer ..."
4,Microsoft,Software Engineer,Entry,"[c, python, java, perl]"


## Building the Recommendation Model

In [ ]:
# Removing all rows with empty skill list in job_descp dataset
index_val = []
for index, skills in enumerate(jobs_desc_df["skills_required"]):
  skills = jobs_desc_df["skills_required"][index]
  if len(skills) == 0 :
    index_val.append(index)

print("Number of blank list: ", len(index_val))
jobs_desc_df = jobs_desc_df.drop(labels=index_val, axis=0)
jobs_desc_df = jobs_desc_df.reset_index(drop=True)

#displaying the dataframe
jobs_desc_df

Number of blank list:  508


,companyName,company_offeredRole,Seniority Level,skills_required
0,Amazon Data Services UK Ltd,Software Engineer,Entry,"[agile, algorithms, master, java, computer sci..."
1,Applied Nanodetectors Ltd,Software Engineer,Mid,"[bachelor, deep learning, algorithm, python, o..."
2,CITADEL ENTERPRISE EUROPE LIMITED,Software Engineer,Entry,"[probability, master, research, phd, computer ..."
3,Microsoft,Software Engineer,Entry,"[c, python, java, perl]"
4,Visa,Software Engineer,Entry,"[javascript, algorithms, master, docker, java,..."
...,...,...,...,...
10913,MetLife Legal Plans,Data Science,Mid,"[professional, tableau, sas, r, matlab, perl, ..."
10914,Dow Jones,Data Science,Mid,"[numpy, algorithms, git, supervised, pandas, m..."
10915,Confidential,Data Science,Mid,"[agile, r, senior, redshift, cloud, bachelor, ..."
10916,Charles Schwab,Data Science,Mid,"[data science, algorithms, git, visualization,..."


***Creating the Enriched Skills column***

This is done witht the help of similarity dataset. 
The model iteratiove overr the dataframe and column and if the JD skills is too short, we will expand the skillset. If the JD is long them there is no point enriching it as it gives the model more skills to match with. 

Here the threshold is 10. If the job description has less than 10 skills then enrich. 

In [ ]:
# Creating a new column
jobs_desc_df["Enriched_Skills"] = None

# Funtion to enrich the skills
def generalising_the_skillset(input_skill_list):

  enriched_skill_list = []

  for skill in input_skill_list:
    # Filtering skills based on input skills
    temp_df = skill_sim_df[skill_sim_df["Skill_1"]== skill]
    # Removing negative values
    temp_df['Simmilarity/Correlation'] = temp_df['Simmilarity/Correlation'].abs()
    # Sorting the values in descending order
    temp_df = temp_df.sort_values('Simmilarity/Correlation', ascending=False)
    # Selecting top 4 skills 
    temp_df = temp_df.iloc[:5]
    # 
    enriched_skill_list.append(list(temp_df['Skill_2']))

  enriched_skill_list = [item for sublist in enriched_skill_list for item in sublist]
  input_skill_list.extend(enriched_skill_list)
  return list(set(input_skill_list))

# Using the funtion on each row in skills_required column 
for index, skills in enumerate(jobs_desc_df["skills_required"]):
  skills = jobs_desc_df["skills_required"][index]
  if len(skills) < 10 :
    skills = generalising_the_skillset(skills)
    jobs_desc_df["Enriched_Skills"][index] = skills
  else:
    jobs_desc_df["Enriched_Skills"][index] = skills

jobs_desc_df

,companyName,company_offeredRole,Seniority Level,skills_required,Enriched_Skills
0,Amazon Data Services UK Ltd,Software Engineer,Entry,"[agile, algorithms, master, java, computer sci...","[agile, algorithms, master, java, computer sci..."
1,Applied Nanodetectors Ltd,Software Engineer,Mid,"[bachelor, deep learning, algorithm, python, o...","[r, django, recommender systems, optimization,..."
2,CITADEL ENTERPRISE EUROPE LIMITED,Software Engineer,Entry,"[probability, master, research, phd, computer ...","[phd, r, microsoft power bi, tuning, computer ..."
3,Microsoft,Software Engineer,Entry,"[c, python, java, perl, c++, embedded, java, p...","[c, r, django, senior software engineer, rest,..."
4,Visa,Software Engineer,Entry,"[javascript, algorithms, master, docker, java,...","[javascript, algorithms, master, docker, java,..."
...,...,...,...,...,...
10913,MetLife Legal Plans,Data Science,Mid,"[professional, tableau, sas, r, matlab, perl, ...","[r, matlab, asp.net, sas, plsql, django, sql s..."
10914,Dow Jones,Data Science,Mid,"[numpy, algorithms, git, supervised, pandas, m...","[numpy, algorithms, git, supervised, pandas, m..."
10915,Confidential,Data Science,Mid,"[agile, r, senior, redshift, cloud, bachelor, ...","[agile, r, senior, redshift, cloud, bachelor, ..."
10916,Charles Schwab,Data Science,Mid,"[data science, algorithms, git, visualization,...","[data science, algorithms, git, visualization,..."


***This step is optional.*** 

***Here the code is finding the intersetion between the enriched skills and the candidate skills.*** 

***As after enriching the skills each JD has atelast 10 skills. The threshold was set that any candidate who's more than 4 skills macthes would be reocmmended forr that posting.***

In [ ]:
# Iterating over candiates profile 
# To find the best candidate for the job
for index, job_req_skill in enumerate(jobs_desc_df["Enriched_Skills"][:10]): 
  job_req_skill = jobs_desc_df["Enriched_Skills"][index]
  count = 0
  
  for indx, cand in enumerate(candidate_df["AllSkills"]):
    skills_per_cand = set(candidate_df["AllSkills"][indx])
    common_skills_btw_JD_cand = skills_per_cand.intersection(set(job_req_skill)) # common skills between candidate skills & JD skills
    if (len(common_skills_btw_JD_cand) > 3):
      print("jobs: ", job_req_skill)
      print("cand: ", skills_per_cand)
      print("common_skills_btw_JD_cand: ", common_skills_btw_JD_cand)
      print("." * 100)
      count += 1
  print("-" * 100)
  print(count)
    #if (len(common_skills_btw_JD_cand) == 0):
      #print("\n\n Candidate skills: \n", skills_per_cand)
      #print("\n\n Common skills after: \n", common_skills_btw_JD_cand)
      #print("Recommended Candidate:", candidate_df.iloc[indx])
    

jobs:  ['agile', 'algorithms', 'master', 'java', 'computer science', 'optimization', 'bachelor', 'data structures', 'python', 'algorithm', 'tdd', 'docker', 'html', 'django', 'node', 'supervised', 'numpy', 'ml', 'master', 'pig', 'bigdata', 'algorithms', 'r', 'ba', 'exploratory', 'android', 'spring', 'hibernate', 'c', 'java ee', 'microsoft power bi', 'splunk', 'cognos', 'data structures', 'postgres', 'cuda', 'snowflake', 'rnn', 'regression', 'segmentation', 'angular', 'rest', 'php', 'oracle', 'kubernetes', 'postgres', 'oracle', 'sql', 'unix', 'gcp', 'django', 'machine learning', 'r', 'flask', 'c++', 'nltk', 'vs', 'powerbi', 'segmentation', 'optimization']
cand:  {'software engineering', 'html', 'php', 'css', 'c', 'big data', 'hadoop', 'aws lambda', 'javascript', 'objective-c', 'xcode', 'java', 'amazon ec2', 'scala', 'matlab', 'eclipse'}
common_skills_btw_JD_cand:  {'html', 'php', 'java', 'c'}
................................................................................................

In [ ]:
# Creating a copy of the job descrription dataset
jobs_desc_df_copy = jobs_desc_df.copy()

In [ ]:
# Create a new dataframe with the follwoing column name
df_ = pd.DataFrame(columns= ["Index_of_job","companyName", "company_offeredRole", 'JD_required_skills', "Candidate_ID", "Candidate_Skills", "Score"])
df_

,Index_of_job,companyName,company_offeredRole,JD_required_skills,Candidate_ID,Candidate_Skills,Score


### Recommending Candidates and storing in new dataframe

***This is same as the code in the previous cell with a slight change.***

Here the code is finding the intersetion between the enriched skills and the candidate skills.

As after enriching the skills each JD has atelast 10 skills. The threshold was set that any candidate who's more than 4 skills macthes would be reocmmended forr that posting.

For every candidate reocmmeded for a job post, a new row is added in the new dataset created with the UserID of the candidate.



In [ ]:
# Iterating over candiates profile 
# To find the best candidate for the job
for index, row in jobs_desc_df_copy.iterrows():
  job_req_skill = jobs_desc_df_copy["Enriched_Skills"][index]

  for indx, cand in enumerate(candidate_df["AllSkills"]):
    skills_per_cand = set(candidate_df["AllSkills"][indx])
    common_skills_btw_JD_cand = skills_per_cand.intersection(set(job_req_skill)) # common skills between candidate skills & JD skills

    if (len(common_skills_btw_JD_cand) > 4):
      score = fuzz.ratio(" ".join(job_req_skill), skills_per_cand)
      userId = candidate_df["userId"][indx]
    
      #print("UserID: ", userId)
      #print("Score: ", score)
      df_ = df_.append({'Index_of_job': index, 'companyName': jobs_desc_df_copy["companyName"][index], 'company_offeredRole': jobs_desc_df_copy["company_offeredRole"][index], 
                        'JD_required_skills': jobs_desc_df_copy["skills_required"][index], 'Candidate_ID': userId, "Candidate_Skills": list(skills_per_cand), "Score" : score}, ignore_index=True)
    
  print("Index: ", index, "DONE.........................................................................")


df_ 


Streaming output truncated to the last 5000 lines.
Index:  5918 DONE.........................................................................
Index:  5919 DONE.........................................................................
Index:  5920 DONE.........................................................................
Index:  5921 DONE.........................................................................
Index:  5922 DONE.........................................................................
Index:  5923 DONE.........................................................................
Index:  5924 DONE.........................................................................
Index:  5925 DONE.........................................................................
Index:  5926 DONE.........................................................................
Index:  5927 DONE.........................................................................
Index:  5928 DONE......................

,Index_of_job,companyName,company_offeredRole,JD_required_skills,Candidate_ID,Candidate_Skills,Score
0,0,Amazon Data Services UK Ltd,Software Engineer,"[agile, algorithms, master, java, computer sci...",207600550,"[c, object-oriented programming (oop), matlab,...",38
1,0,Amazon Data Services UK Ltd,Software Engineer,"[agile, algorithms, master, java, computer sci...",44528135,"[c, machine learning, natural language process...",35
2,0,Amazon Data Services UK Ltd,Software Engineer,"[agile, algorithms, master, java, computer sci...",440700307,"[aws sagemaker, pytorch, software development,...",30
3,1,Applied Nanodetectors Ltd,Software Engineer,"[bachelor, deep learning, algorithm, python, o...",48513720,"[c, machine learning, object-oriented programm...",35
4,1,Applied Nanodetectors Ltd,Software Engineer,"[bachelor, deep learning, algorithm, python, o...",72847910,"[c, machine learning, matlab, java, python, cu...",38
...,...,...,...,...,...,...,...
202328,10917,Opanga Networks,Data Science,"[data science, numpy, tableau, r, visualizatio...",307725385,"[data visualization, r, statistics, python , s...",41
202329,10917,Opanga Networks,Data Science,"[data science, numpy, tableau, r, visualizatio...",557552884,"[numpy, data visualization, pandas, python , s...",44
202330,10917,Opanga Networks,Data Science,"[data science, numpy, tableau, r, visualizatio...",400680128,"[data visualization, machine learning, r, deep...",43
202331,10917,Opanga Networks,Data Science,"[data science, numpy, tableau, r, visualizatio...",651477149,"[data visualization, tensorflow, r, recommende...",37


In [ ]:
# saving all recommendation mades in a csv
df_.to_csv("All_Recommendations.csv")

## Displaying Result

### Software Engineering

In [ ]:
# Based on index value of a company, display desired candidates and there profiles
def recommend_candidates(index_value):
  rows = df_[df_["Index_of_job"]== index_value]
  rows = rows.sort_values('Score', ascending=False)
  rows = rows.iloc[:10] # Recommend top 10
  print("_" * 60,"RECOMMENDED CANDIDATES", "_" * 60)
  print("\n\n")
  list_of_candidates = list(rows["Candidate_ID"])
  print("List of userID's recommended: ", list_of_candidates)
  print("." * 190)
  print("\n\n\n The ranked Candidates are based on matching skills: \n")
  print("-" * 190)
  display(rows)
  #for user in list_of_candidates:
  cand_profile_df = candidate_full_profile[candidate_full_profile["userId"].isin(list_of_candidates)]
  print("\n")
  print("-" * 190)
  print("\n\n")
  print("\n Candidates Profiles: ")
  print("-" * 190)
  display(cand_profile_df)

In [ ]:
# Type in the index value of a company to display desired candidates
recommend_candidates(3)

____________________________________________________________ RECOMMENDED CANDIDATES ____________________________________________________________



List of userID's recommended:  [417666743, 163866689, 371916239, 646990176, 445502985, 646990176, 372961138, 72615128, 48513720, 72847910]
..............................................................................................................................................................................................



 The ranked Candidates are based on matching skills: 

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Index_of_job,companyName,company_offeredRole,JD_required_skills,Candidate_ID,Candidate_Skills,Score
41,3,Microsoft,Software Engineer,"[c, python, java, perl, c++, embedded, java, p...",417666743,"[numpy, c, r, django, mongodb, python, algorit...",41
51,3,Microsoft,Software Engineer,"[c, python, java, perl, c++, embedded, java, p...",163866689,"[c, matlab, software development, objective-c,...",39
39,3,Microsoft,Software Engineer,"[c, python, java, perl, c++, embedded, java, p...",371916239,"[c, pytorch, software development, front-end d...",39
58,3,Microsoft,Software Engineer,"[c, python, java, perl, c++, embedded, java, p...",646990176,"[numpy, c, r, microsoft power bi, neural netwo...",38
47,3,Microsoft,Software Engineer,"[c, python, java, perl, c++, embedded, java, p...",445502985,"[c, matlab, software development, apache kafka...",38
56,3,Microsoft,Software Engineer,"[c, python, java, perl, c++, embedded, java, p...",646990176,"[numpy, c, r, microsoft power bi, neural netwo...",38
46,3,Microsoft,Software Engineer,"[c, python, java, perl, c++, embedded, java, p...",372961138,"[c, node.js, data visualization, machine learn...",37
48,3,Microsoft,Software Engineer,"[c, python, java, perl, c++, embedded, java, p...",72615128,"[c, natural language processing, machine learn...",36
37,3,Microsoft,Software Engineer,"[c, python, java, perl, c++, embedded, java, p...",48513720,"[c, machine learning, object-oriented programm...",35
38,3,Microsoft,Software Engineer,"[c, python, java, perl, c++, embedded, java, p...",72847910,"[c, machine learning, matlab, java, python, cu...",35




----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------




 Candidates Profiles: 
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,userId,Name,Current_Company,Job_Title,Job_Date_Range,Company_2,Job_Title2,Job_Date_Range2,Job_Description_2,University,Degree,Degree_Name,AllSkills
7,48513720,Yogeetha Sundaram,Netflix,Senior Software Engineer,Feb 2017 – Present,Yahoo! Inc.,"Tech Yahoo!, Software Development Engineer",Feb 2015 – Jan 2017,NaN,PSG College of Technology,Master's,Software Engineering,"Python, Shell Scripting, Algorithms, Web Servi..."
8,72847910,Ziang Ding,Microsoft,Senior Software Engineer,Sep 2020 – Present,Microsoft,SDE II,Apr 2018 – Sep 2020,NaN,Purdue University,PhD,Computer Science,"C++, Matlab, Scientific Visualization, 3D grap..."
11,371916239,Shuya Lin,LinkedIn,Software Engineer Intern,Sep 2021 – Present,Unity Technologies,Software Engineer Intern,May 2021 – Aug 2021,NaN,University of Pennsylvania,Master's,Computer Science,"Python (Programming Language), Machine Learnin..."
32,417666743,Yash Katariya,Google,Software Engineer,Jan 2019 – Present,Google,Intern,May 2018 – Aug 2018,* Worked with the Eager team to create new fea...,North Carolina State University,Master's,Computer Science,"Python, Django, Keras, Deep Learning, Data Str..."
64,372961138,Jamal Rahman,Captify,Machine Learning Engineer,Jan 2021 – Present,IBM,Data Scientist,Nov 2019 – Dec 2020,Built Cloud Native Machine Learning MVPs using...,University of Birmingham,Master's,Computer Science,"Machine Learning, Python, Software Engineering..."
67,445502985,Weng Hoe Chen,WorldRemit,Machine Learning Engineer,Nov 2019 – Present,THG,Data Scientist,Jul 2018 – Nov 2019,- Image similarity based recommender system fo...,UCL,Master's,NaN,"Python, Machine Learning, Big Data, Integratio..."
70,72615128,Kashyap Popat,Facebook AI,Machine Learning Engineer,Dec 2019 – Present,Max Planck Institute for Informatics,PHD Candidate,Sep 2015 – Nov 2019,"Dissertation Topic: ""Credibility Analysis of T...",IIT Bombay,Not Given,Computer Science,"C, Machine Learning, C++, Data Structures, Pro..."
100,163866689,Kimisha Mody,Apple,Machine Learning Engineer,Feb 2018 – Present,Georgia Institute of Technology,Graduate Teaching Assistant,Aug 2017 – Dec 2017,Knowledge-Based Artificial Intelligence,Georgia Institute of Technology,Master's,Computer Science,"Java, C++, SQL, Machine Learning, Data Analysi..."
130,646990176,Diganth Dinesh Mahale,Internet Brands,Associate Data Scientist,Sep 2021 – Present,UTDSOM Investment Corp.,Data Science Intern,Aug 2020 – May 2021,• Predicted quarterly stock prices for 150+ co...,The University of Texas at Dallas,Not Given,NaN,"Python (Programming Language), MySQL, Tableau,..."
160,646990176,Diganth Dinesh Mahale,Internet Brands,Associate Data Scientist,Sep 2021 – Present,UTDSOM Investment Corp.,Data Science Intern,Aug 2020 – May 2021,• Predicted quarterly stock prices for 150+ co...,The University of Texas at Dallas,Not Given,NaN,"Python (Programming Language), MySQL, Tableau,..."


In [ ]:
# Type in the index value of a company to display desired candidates
recommend_candidates(0)

____________________________________________________________ RECOMMENDED CANDIDATES ____________________________________________________________



List of userID's recommended:  [207600550, 44528135, 440700307]
..............................................................................................................................................................................................



 The ranked Candidates are based on matching skills: 

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Index_of_job,companyName,company_offeredRole,JD_required_skills,Candidate_ID,Candidate_Skills,Score
0,0,Amazon Data Services UK Ltd,Software Engineer,"[agile, algorithms, master, java, computer sci...",207600550,"[c, object-oriented programming (oop), matlab,...",38
1,0,Amazon Data Services UK Ltd,Software Engineer,"[agile, algorithms, master, java, computer sci...",44528135,"[c, machine learning, natural language process...",35
2,0,Amazon Data Services UK Ltd,Software Engineer,"[agile, algorithms, master, java, computer sci...",440700307,"[aws sagemaker, pytorch, software development,...",30




----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------




 Candidates Profiles: 
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,userId,Name,Current_Company,Job_Title,Job_Date_Range,Company_2,Job_Title2,Job_Date_Range2,Job_Description_2,University,Degree,Degree_Name,AllSkills
3,207600550,Fan Di,Google,Senior Software Engineer,Jun 2018 – Present,MathWorks,Software Engineer Intern,May 2017 – May 2018,Jan 2018 - May 2018 (Part-time 20 hrs/week in ...,Harvard University,Master's,Landscape Architecture,"Java, Algorithms, Python, Data Structures, Art..."
4,44528135,Neha Gupta,Google,Senior Software Engineer,Nov 2019 – Present,Google,Software Engineer,Feb 2015 – Nov 2019,Adwords Frontend Software Engineer,Georgia Institute of Technology,Bachelor’s,Computer Science,"Machine Learning, C, Linux, Artificial Intelli..."
62,440700307,Marcello De Bernardi,Expedia Group,Machine Learning Engineer III,Nov 2021 – Present,Amazon Web Services (AWS),Staff Consultant (Data Science),Jul 2021 – Oct 2021,"Working with AWS customers to design, implemen...",University of Oxford,Master's,Computer Science,"Machine Learning, Amazon Web Services (AWS), P..."


In [ ]:
# Type in the index value of a company to display desired candidates
recommend_candidates(5)

____________________________________________________________ RECOMMENDED CANDIDATES ____________________________________________________________



List of userID's recommended:  [440700307, 417666743, 646990176, 646990176, 775328907, 344063232, 480306439, 584494201]
..............................................................................................................................................................................................



 The ranked Candidates are based on matching skills: 

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Index_of_job,companyName,company_offeredRole,JD_required_skills,Candidate_ID,Candidate_Skills,Score
85,5,Moley robotics,Software Engineer,"[algorithms, agile, computer vision, computer ...",440700307,"[aws sagemaker, pytorch, software development,...",46
82,5,Moley robotics,Software Engineer,"[algorithms, agile, computer vision, computer ...",417666743,"[numpy, c, r, django, mongodb, python, algorit...",43
87,5,Moley robotics,Software Engineer,"[algorithms, agile, computer vision, computer ...",646990176,"[numpy, c, r, microsoft power bi, neural netwo...",43
88,5,Moley robotics,Software Engineer,"[algorithms, agile, computer vision, computer ...",646990176,"[numpy, c, r, microsoft power bi, neural netwo...",43
84,5,Moley robotics,Software Engineer,"[algorithms, agile, computer vision, computer ...",775328907,"[numpy, genetic algorithm, c, r, matlab, githu...",40
86,5,Moley robotics,Software Engineer,"[algorithms, agile, computer vision, computer ...",344063232,"[c, data visualization, pytorch, matlab, compu...",40
83,5,Moley robotics,Software Engineer,"[algorithms, agile, computer vision, computer ...",480306439,"[numpy, c, matlab, sensors, deep learning, jav...",39
89,5,Moley robotics,Software Engineer,"[algorithms, agile, computer vision, computer ...",584494201,"[numpy, c, r, matlab, computer science, python...",39




----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------




 Candidates Profiles: 
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,userId,Name,Current_Company,Job_Title,Job_Date_Range,Company_2,Job_Title2,Job_Date_Range2,Job_Description_2,University,Degree,Degree_Name,AllSkills
32,417666743,Yash Katariya,Google,Software Engineer,Jan 2019 – Present,Google,Intern,May 2018 – Aug 2018,* Worked with the Eager team to create new fea...,North Carolina State University,Master's,Computer Science,"Python, Django, Keras, Deep Learning, Data Str..."
39,480306439,Prerit Jain,Intel Corporation,Software Engineer,Mar 2021 – Present,Shoptaki,Data Science Intern,Dec 2020 – Mar 2021,Shoptaki is an evolutionary startup that is de...,Virginia Tech,Master's,Computer Engineering,"Python (Programming Language), Java, Matlab, E..."
48,775328907,Almpes K.,Cisco,Software Engineer,Sep 2020 – Present,Pizza Hut Restaurants UK,Guest Server | Kitchen Staff Member,Aug 2017 – Oct 2019,Worked part-time both FOH and BOH in one of th...,University of Strathclyde,Master's,Advanced Computer Science with Big Data.,"Python (Programming Language), C, Problem Solv..."
62,440700307,Marcello De Bernardi,Expedia Group,Machine Learning Engineer III,Nov 2021 – Present,Amazon Web Services (AWS),Staff Consultant (Data Science),Jul 2021 – Oct 2021,"Working with AWS customers to design, implemen...",University of Oxford,Master's,Computer Science,"Machine Learning, Amazon Web Services (AWS), P..."
73,344063232,Aishwarya Vellappally,CGG Data Management,Machine Learning Engineer,Mar 2021 – Present,TutorMe,Tutor,Oct 2019 – Present,Tutoring in Basic and Advanced Mathematics and...,UCL,Master's,Computer Science,"Python (Programming Language), Machine Learnin..."
130,646990176,Diganth Dinesh Mahale,Internet Brands,Associate Data Scientist,Sep 2021 – Present,UTDSOM Investment Corp.,Data Science Intern,Aug 2020 – May 2021,• Predicted quarterly stock prices for 150+ co...,The University of Texas at Dallas,Not Given,NaN,"Python (Programming Language), MySQL, Tableau,..."
160,646990176,Diganth Dinesh Mahale,Internet Brands,Associate Data Scientist,Sep 2021 – Present,UTDSOM Investment Corp.,Data Science Intern,Aug 2020 – May 2021,• Predicted quarterly stock prices for 150+ co...,The University of Texas at Dallas,Not Given,NaN,"Python (Programming Language), MySQL, Tableau,..."
198,584494201,Peter Gilbert,bp,Data Scientist,Sep 2020 – Present,Morgan Stanley,Summer Analyst,Jun 2019 – Aug 2019,"Worked in the Fund Services department, in the...",University of Exeter,Master's,Computer Science,"Machine Learning, Amazon Web Services (AWS), C..."


### Machine Learning Engineers

In [ ]:
# Type in the index value of a company to display desired candidates
recommend_candidates(5000)

____________________________________________________________ RECOMMENDED CANDIDATES ____________________________________________________________



List of userID's recommended:  [276535467, 208706220, 536129795, 308377708, 616940509, 616940509, 616940509, 445502985, 344063232, 558034209]
..............................................................................................................................................................................................



 The ranked Candidates are based on matching skills: 

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Index_of_job,companyName,company_offeredRole,JD_required_skills,Candidate_ID,Candidate_Skills,Score
74869,5000,Niantic,Machine Learning Engineer,"[geospatial, data science, experimental design...",276535467,"[data science & statistics, jupyter, machine l...",44
74868,5000,Niantic,Machine Learning Engineer,"[geospatial, data science, experimental design...",208706220,"[pytorch, matlab, bayesian statistics, compute...",42
74864,5000,Niantic,Machine Learning Engineer,"[geospatial, data science, experimental design...",536129795,"[calculus, machine learning, natural language ...",39
74873,5000,Niantic,Machine Learning Engineer,"[geospatial, data science, experimental design...",308377708,"[machine learning, matlab, statistics, python ...",39
74870,5000,Niantic,Machine Learning Engineer,"[geospatial, data science, experimental design...",616940509,"[r, machine learning, statistics, sql, python,...",37
74871,5000,Niantic,Machine Learning Engineer,"[geospatial, data science, experimental design...",616940509,"[r, machine learning, statistics, sql, python,...",37
74872,5000,Niantic,Machine Learning Engineer,"[geospatial, data science, experimental design...",616940509,"[r, machine learning, statistics, sql, python,...",37
74865,5000,Niantic,Machine Learning Engineer,"[geospatial, data science, experimental design...",445502985,"[c, matlab, software development, apache kafka...",36
74866,5000,Niantic,Machine Learning Engineer,"[geospatial, data science, experimental design...",344063232,"[c, data visualization, pytorch, matlab, compu...",36
74867,5000,Niantic,Machine Learning Engineer,"[geospatial, data science, experimental design...",558034209,"[machine learning, javascript, statistics, sql...",36




----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------




 Candidates Profiles: 
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,userId,Name,Current_Company,Job_Title,Job_Date_Range,Company_2,Job_Title2,Job_Date_Range2,Job_Description_2,University,Degree,Degree_Name,AllSkills
63,536129795,Vasilis Papastefanopoulos,Programmai,Machine Learning Engineer,Nov 2018 – Present,The Francis Crick Institute,Research Intern,Jun 2018 – Sep 2018,- Protein function prediction using Deep Lear...,UCL,Master's,Machine Learning,"Python, Machine Learning, Statistics, Programm..."
67,445502985,Weng Hoe Chen,WorldRemit,Machine Learning Engineer,Nov 2019 – Present,THG,Data Scientist,Jul 2018 – Nov 2019,- Image similarity based recommender system fo...,UCL,Master's,NaN,"Python, Machine Learning, Big Data, Integratio..."
73,344063232,Aishwarya Vellappally,CGG Data Management,Machine Learning Engineer,Mar 2021 – Present,TutorMe,Tutor,Oct 2019 – Present,Tutoring in Basic and Advanced Mathematics and...,UCL,Master's,Computer Science,"Python (Programming Language), Machine Learnin..."
76,558034209,Tomasz Bartkowiak,QuantumBlack,"Senior Consultant, Machine Learning Engineer",Jul 2021 – Present,QuantumBlack,"Consultant, Machine Learning Engineer",Jan 2020 – Jun 2021,NaN,Imperial College London,Master's,Computer Science,"C/C++, Python, Machine Learning, Statistics, M..."
115,208706220,David Bourgin,Spotify,Machine Learning Engineer,Jan 2020 – Present,Princeton University,Postdoctoral Researcher,Aug 2018 – Nov 2018,Postdoc in the computational cognitive science...,UC Berkeley,PhD,Cognitive Science,"Teaching, Matlab, Statistics, Research, Data A..."
117,276535467,Tina Rezvanian,Adobe,Machine Learning Engineer IV,Sep 2019 – Present,Northeastern University,Ph.D. Industrial Engineering,Sep 2013 – Sep 2019,Forecasting Spatio-Temporal Emergency Demand a...,Northeastern University,Not Given,Industrial Engineering and Operations Research...,"Data Science, Operations Research, Python, Mac..."
123,616940509,Matthew Clarke-Williams,Tonic Analytics,Data Scientist Intern,Jul 2018 – Sep 2018,NaN,NaN,NaN,NaN,University of Bath,Master's,Data Science with Placement Year,"Data Analysis, Python, SQL, Statistics, Machin..."
143,616940509,Matthew Clarke-Williams,Tonic Analytics,Data Scientist Intern,Jul 2018 – Sep 2018,NaN,NaN,NaN,NaN,University of Bath,Master's,Data Science with Placement Year,"Data Analysis, Python, SQL, Statistics, Machin..."
181,616940509,Matthew Clarke-Williams,Tonic Analytics,Data Scientist Intern,Jul 2018 – Sep 2018,NaN,NaN,NaN,NaN,University of Bath,Master's,Data Science with Placement Year,"Data Analysis, Python, SQL, Statistics, Machin..."
201,308377708,Wai Kan (Kenny) Fong,Marks and Spencer,Senior Data Scientist,Mar 2020 – Present,Expedia Group,Data Scientist,Feb 2017 – Feb 2020,NaN,University of Oxford,Not Given,MEng Engineering Science,"Python, Machine Learning, Bayesian inference, ..."


### Data Scientist

In [ ]:
# Type in the index value of a company to display desired candidates
recommend_candidates(10000)

____________________________________________________________ RECOMMENDED CANDIDATES ____________________________________________________________



List of userID's recommended:  [383351669, 375022118, 646990176, 646990176, 58423523, 400680128, 400680128, 400680128]
..............................................................................................................................................................................................



 The ranked Candidates are based on matching skills: 

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Index_of_job,companyName,company_offeredRole,JD_required_skills,Candidate_ID,Candidate_Skills,Score
183476,10000,Intelligent Data Technologies Limited (Lucid),Data Science,"[data science, ml, app development, research, ...",383351669,"[data visualization, pytorch, r, bayesian stat...",41
183477,10000,Intelligent Data Technologies Limited (Lucid),Data Science,"[data science, ml, app development, research, ...",375022118,"[numpy, big data optimization, pytorch, r, mat...",40
183480,10000,Intelligent Data Technologies Limited (Lucid),Data Science,"[data science, ml, app development, research, ...",646990176,"[numpy, c, r, microsoft power bi, neural netwo...",40
183482,10000,Intelligent Data Technologies Limited (Lucid),Data Science,"[data science, ml, app development, research, ...",646990176,"[numpy, c, r, microsoft power bi, neural netwo...",40
183478,10000,Intelligent Data Technologies Limited (Lucid),Data Science,"[data science, ml, app development, research, ...",58423523,"[numpy, c, r, artificial neural networks, jque...",37
183479,10000,Intelligent Data Technologies Limited (Lucid),Data Science,"[data science, ml, app development, research, ...",400680128,"[data visualization, machine learning, r, deep...",35
183481,10000,Intelligent Data Technologies Limited (Lucid),Data Science,"[data science, ml, app development, research, ...",400680128,"[data visualization, machine learning, r, deep...",35
183483,10000,Intelligent Data Technologies Limited (Lucid),Data Science,"[data science, ml, app development, research, ...",400680128,"[data visualization, machine learning, r, deep...",35




----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------




 Candidates Profiles: 
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,userId,Name,Current_Company,Job_Title,Job_Date_Range,Company_2,Job_Title2,Job_Date_Range2,Job_Description_2,University,Degree,Degree_Name,AllSkills
88,383351669,Ziwei Ye,Spotify,Machine Learning Engineer Intern,Jul 2021 – Present,Spotify,Machine Learning Engineer Intern,Jul 2020 – Jan 2021,"Podcast Recommendation, Personalization Platfo...",Rochester Institute of Technology,Master's,Computational and Applied Mathematics,"Google Cloud Platform (GCP), Scala, Statistica..."
110,375022118,Mandana Saebi,Apple,Machine Learning Engineer,Sep 2021 – Present,Microsoft,Data Science Intern,Jun 2021 – Aug 2021,Developed machine learning models for predicti...,University of Notre Dame,PhD,Computer Science,"Machine Learning, Data Science, Python (Progra..."
116,58423523,Preetham M.,Spotify,Machine Learning Engineer,May 2016 – Present,Laboratory for Analytic Sciences,Research Assistant,May 2015 – Dec 2015,Text correction in optical character recognize...,North Carolina State University,Master's,Computer Science (Data Science track),"Python, Apache Spark, Big Data, Machine Learni..."
121,400680128,Yue (Harper) Han,Monitor Deloitte (ex Monitor Group),Artificial Intelligence Data Scientist Intern,May 2019 – Sep 2019,Southwest Jiaotong University,Graduation Project - Application of Data Minin...,Dec 2017 – Jun 2018,NaN,University of Warwick - Warwick Business School,Master's,Business Analytics,"Data Analytics, Machine Learning, Python, Rese..."
130,646990176,Diganth Dinesh Mahale,Internet Brands,Associate Data Scientist,Sep 2021 – Present,UTDSOM Investment Corp.,Data Science Intern,Aug 2020 – May 2021,• Predicted quarterly stock prices for 150+ co...,The University of Texas at Dallas,Not Given,NaN,"Python (Programming Language), MySQL, Tableau,..."
141,400680128,Yue (Harper) Han,Monitor Deloitte (ex Monitor Group),Artificial Intelligence Data Scientist Intern,May 2019 – Sep 2019,Southwest Jiaotong University,Graduation Project - Application of Data Minin...,Dec 2017 – Jun 2018,NaN,University of Warwick - Warwick Business School,Master's,Business Analytics,"Data Analytics, Machine Learning, Python, Rese..."
160,646990176,Diganth Dinesh Mahale,Internet Brands,Associate Data Scientist,Sep 2021 – Present,UTDSOM Investment Corp.,Data Science Intern,Aug 2020 – May 2021,• Predicted quarterly stock prices for 150+ co...,The University of Texas at Dallas,Not Given,NaN,"Python (Programming Language), MySQL, Tableau,..."
179,400680128,Yue (Harper) Han,Monitor Deloitte (ex Monitor Group),Artificial Intelligence Data Scientist Intern,May 2019 – Sep 2019,Southwest Jiaotong University,Graduation Project - Application of Data Minin...,Dec 2017 – Jun 2018,NaN,University of Warwick - Warwick Business School,Master's,Business Analytics,"Data Analytics, Machine Learning, Python, Rese..."
